# Channel Mean + MLP

This is an implementation of Jain et. al. on our dataset

## Reference
Jain, Prakhar, Shubham Bauskar, and Manasi Gyanchandani. "Neural network based non‐invasive method to detect anemia from images of eye conjunctiva." International Journal of Imaging Systems and Technology 30.1 (2020): 112-125.


## Imports

In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas
from tqdm import tqdm

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

## Load Data

In [21]:
y_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_forniceal_italy.npy") #/30
X_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_forniceal_italy.npy")/255

In [22]:
y_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_forniceal_india.npy") #/30
X_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_forniceal_india.npy")/255

In [143]:
## temp
X_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_proc_italy.npy")/255
X_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_proc_india.npy")/255

y_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_base_italy.npy")
y_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_base_india.npy")

## Data Preparation

In [23]:
X_npy = np.concatenate((X_in, X_it), axis=0)
y_npy = np.concatenate((y_in, y_it), axis=0)/20
X_npy.shape, y_npy.shape

((212, 640, 480, 3), (212,))

In [24]:
y_npy[y_npy > 0.75] = 1
y_npy[y_npy < 1] = 0
y_npy

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 1.], dtype=float32)

## Channel Mean Computation

In [28]:
X_r = X_npy[:,:,:,2].mean(axis=(1,2))
X_g = X_npy[:,:,:,1].mean(axis=(1,2))
X_input = np.array([X_r,  X_g]).T

# X_r = X_npy[:,:,:,0].mean(axis=(1,2))
# X_g = X_npy[:,:,:,0].mean(axis=(1,2))
# X_input = np.array([X_r,  X_g]).T

In [29]:
# X[X==1] = 0
# plt.imshow(X[10])

## ANN/MLP

In [30]:
from torch.autograd import Variable
# Ref: https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/

In [31]:
train_split=0.75
val_split = 0.10
test_split = 1 - train_split - val_split
batch_size = 3
epochs = 10

# X=X[:,:,:,0]
# X_in = torch.tensor(X).float().unsqueeze(1).float()
X = X_input
X_input = torch.from_numpy(X).float() #.permute(0,3,2,1)
# X_in.shape()

y_input = torch.tensor(y_npy).float().unsqueeze(1)

print(X_input.shape, y_input.shape)

torch.Size([212, 2]) torch.Size([212, 1])


In [32]:
from sklearn.model_selection import train_test_split
X_use, X_test, y_use,  y_test = train_test_split(X_input, y_input, test_size=test_split)
X_train, X_val, y_train, y_val = train_test_split(X_use, y_use, test_size=val_split/(val_split+train_split))

In [33]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

(torch.Size([158, 2]),
 torch.Size([22, 2]),
 torch.Size([32, 2]),
 torch.Size([158, 1]),
 torch.Size([22, 1]),
 torch.Size([32, 1]))

## Model Generation

Simple model with 2-6-3-1 structure. Activation sigmoid for 0-1 outputs scaling.

In [34]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 6)
        self.fc2 = nn.Linear(6, 3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
#         x = F.relu(self.fc2(x))
        return x

## Training

In [35]:
import torch.optim as optim

net = Net()

criterion = nn.BCELoss()
#nn.L1Loss()
#nn.BCEWithLogitsLoss()
#nn.MSELoss()

optimizer = optim.Adam(net.parameters(), lr=0.01)
#optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#
#
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

## Minor Validation

In [36]:
y_test.detach().numpy().flatten(), net(X_test).detach().numpy().flatten(), criterion(net(X_test), torch.Tensor(y_test)),

C:\Users\manas\AppData\Roaming\Python\Python39\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
        1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       dtype=float32),
 array([0.43688282, 0.43689838, 0.4348906 , 0.43534613, 0.43498847,
        0.43490046, 0.4353085 , 0.43516636, 0.43470535, 0.43501237,
        0.43485352, 0.43469614, 0.43511823, 0.43563506, 0.4350688 ,
        0.43519723, 0.43482712, 0.4349835 , 0.4348093 , 0.43576822,
        0.43500304, 0.4347538 , 0.43457368, 0.43689653, 0.43458557,
        0.4351195 , 0.43502465, 0.43519825, 0.4353249 , 0.4368975 ,
        0.4350723 , 0.43462706], dtype=float32),
 tensor(0.6611, grad_fn=<BinaryCrossEntropyBackward0>))

## Full Scale Validation

In [37]:
batch_size = 10
epochs = 20

# Remove later
net = Net()

net.train()

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    iters = (len(X_train)//batch_size)+1
    
    for i in range((len(X_train)//batch_size)+1):
        # get the inputs; data is a list of [inputs, labels]
        
        try:
            inputs = X_train[(i*batch_size):(i+1)*batch_size]
            labels = y_train[(i*batch_size):(i+1)*batch_size]
        except:
            inputs = X_train[(i*batch_size):]
            labels = y_train[(i*batch_size):]
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
#         print(outputs.shape, labels.shape, outputs.dtype, labels.dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
#     with torch.no_grad():
    y_pred_val = net(Variable(X_val))
    val_loss = criterion(y_pred_val, Variable(torch.Tensor(y_val)))
    validation_loss += val_loss.item()
    
    if epoch %20 == 0 or epoch == epochs-1:
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / iters:.9f}, val_loss: {val_loss.item():.3f}')

print(y_pred_val)
#     running_loss = 0.0
#     validation_loss = 0.0
    #scheduler.step()

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 272.90it/s]

[1,    16] loss: 0.610886823, val_loss: 0.650
[20,    16] loss: 0.610886823, val_loss: 0.650
tensor([[0.4380],
        [0.4381],
        [0.4107],
        [0.4381],
        [0.4381],
        [0.4103],
        [0.4380],
        [0.4379],
        [0.4380],
        [0.4106],
        [0.4380],
        [0.4101],
        [0.4380],
        [0.4381],
        [0.4378],
        [0.4108],
        [0.4379],
        [0.4382],
        [0.4380],
        [0.4363],
        [0.4381],
        [0.4382]], grad_fn=<SigmoidBackward0>)


In [38]:
# torch.save(net.state_dict(), 'model_weights.pth')
# model.load_state_dict(torch.load('model_weights.pth'))
# model.eval()

In [39]:
y_hat = net(X_test).detach().numpy()
y_test = y_test.detach().numpy()
y_hat[y_hat >= 0.75] = 0
y_hat[y_hat > 0] = 1
y_hat, y_test

(array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32),
 array([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.]], dtype=float32))

In [40]:
from sklearn.metrics import classification_report

In [41]:
print(classification_report(1-y_test, y_hat))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.66      1.00      0.79        21

    accuracy                           0.66        32
   macro avg       0.33      0.50      0.40        32
weighted avg       0.43      0.66      0.52        32



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
# y_hat.shape, y_test.shape
criterion(y_hat, y_test)

In [124]:
nn.L1Loss()(y_hat*30, y_test*30), nn.MSELoss()(y_hat*20, y_test*20)

TypeError: 'int' object is not callable

In [40]:
net(X_test[5].unsqueeze(0))

tensor([[0.5036]], grad_fn=<SigmoidBackward0>)

In [41]:
#plt.imshow(X_test[2].permute(2,1,0))

In [42]:
# for i in range(len(X_test)):
#     print(f'Image: {i+1}, Hb: {y_test[i].item()}, Pred: {y_hat[i].item()}')
#     plt.imshow(X_test[i].permute(2,1,0))
#     plt.figure()

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
reg = LogisticRegression()
reg.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0

In [441]:
y_val_pred = reg.predict(X_val)
y_val, y_val_pred

(tensor([[0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.]]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.], dtype=float32))